In [28]:
import pickle
import urllib.request
import requests
from os.path import join,isfile
from os import listdir
from tqdm import tqdm
import urllib
import multiprocessing as mp
import re

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

In [29]:
with open('qids.pickle', 'rb') as f:
    graph = pickle.load(f)

In [30]:
len(graph)

7653323

In [38]:
mini_graph = graph[:1000]
print(len(mini_graph))

1000


In [45]:
response = requests.get(f"https://www.wikidata.org/wiki/Special:EntityData/Q4378.ttl")

In [37]:
response.status_code == 200
# print (response.content)
# response.encoding = 'utf-8'

True

In [49]:
text = response.text
print(text)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix ontolex: <http://www.w3.org/ns/lemon/ontolex#> .
@prefix dct: <http://purl.org/dc/terms/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix wikibase: <http://wikiba.se/ontology#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix schema: <http://schema.org/> .
@prefix cc: <http://creativecommons.org/ns#> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix wd: <http://www.wikidata.org/entity/> .
@prefix data: <https://www.wikidata.org/wiki/Special:EntityData/> .
@prefix s: <http://www.wikidata.org/entity/statement/> .
@prefix ref: <http://www.wikidata.org/reference/> .
@prefix v: <http://www.wikidata.org/value/> .
@prefix wdt: <http://www.wikidata.org/prop/direct/> .
@prefix wdtn: <http://www.wikidata.org/prop/direct-normalized/> .
@pr

In [27]:
with open(r"dump/Q4378.ttl", "w" , encoding='utf-8') as file:
    file.write(response.text)

In [ ]:
print(response.text[:1000])

In [ ]:
mini_graph[0]

In [ ]:
counter_for_files('dump')

In [5]:
%%time
dic = {}
for i in tqdm(range(len(mini_graph))):
    if f'dump\\{mini_graph[i]}.txt' in counter_for_files('dump'):
        continue
    with requests.get(f"https://www.wikidata.org/wiki/Special:EntityData/{mini_graph[i]}.ttl", stream=True, proxies=urllib.request.getproxies()) as r:
        dic[mini_graph[i]] = r.status_code
        with open(f"dump//{mini_graph[i]}.txt", "w" , encoding='utf-8') as file:
            file.write(r.text)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [08:57<00:00,  1.07s/it]


Wall time: 8min 57s


In [ ]:
dic

In [42]:
my_file = open('Logging.txt', 'w')
my_file.close()

In [43]:
%%file request_file.py


import pickle
import urllib.request
import requests
from os.path import join,isfile
from os import listdir
import urllib

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
        
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles



def request_func(ID):
    if f'dump\\{ID}.ttl' in counter_for_files('dump'):
        return None
    with requests.get(f"https://www.wikidata.org/wiki/Special:EntityData/{ID}.ttl", 
                      stream=True, proxies=urllib.request.getproxies()) as r:
        if r.status_code != 200:
            with open(f"Logging.txt", "a" , encoding='utf-8') as file:
                file.write(f'{ID} :  {r.status_code}\n')
            return None
        with open(f"dump//{ID}.ttl", "w" , encoding='utf-8') as file:
            file.write(r.text)
        return None

Overwriting request_file.py


In [44]:
%%time
import request_file
pool = mp.Pool(processes=mp.cpu_count())
results = pool.map(request_file.request_func, mini_graph)

Wall time: 2min
